In [1]:
#Celda para librerías
import sklearn as sk
import sys
print("Python executable:", sys.executable)

import matplotlib as plt
import matplotlib.pyplot as plt

import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score

import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

#XGBoost
from xgboost import XGBClassifier



from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score


import json

import warnings
warnings.filterwarnings('ignore')

print(np.__version__)
from ydata_profiling import ProfileReport
import os
print("Directorio actual:", os.getcwd())

# Have all columns appear when dataframes are displayed.
pd.set_option('display.max_columns', None) 
# Have 100 rows appear when a dataframe is displayed
pd.set_option('display.max_rows', 500)
# Display dimensions whenever a dataframe is printed out.
pd.set_option('display.show_dimensions', True)

#Importando los datos

#app_train_def = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v1.csv')
#app_train_def_2 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v2.csv')
#app_train_def_3 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v3.csv')
#app_train_def_4 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v4.csv')
#app_train_def_5 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v5.csv')
app_train_def_7 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v7.csv')
app_train_def_7_v2 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/app_def_v7_menor_dimension.csv')

#print(app_train_def_7.info())
#print(app_train_def_7.shape)

Python executable: /home/yeray/miniconda3/envs/tfg-py3.12/bin/python
1.26.4
Directorio actual: /home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS


In [2]:
X = app_train_def_7_v2.drop(columns = ['TARGET'])
y = app_train_def_7_v2['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train.columns = X_train.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)
X_test.columns = X_test.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)

In [ ]:
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 900, 1800, step=50),
        "max_bin": trial.suggest_int("max_bin", 200, 400, step=10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.03, log=True),
        "max_depth": trial.suggest_int("max_depth", 5, 10),
        "gamma": trial.suggest_float("gamma", 0, 3),
        "min_child_weight": trial.suggest_int("min_child_weight", 12, 24),
        "subsample": trial.suggest_float("subsample", 0.6, 0.95),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 0.8),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 1.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "tree_method": trial.suggest_categorical("tree_method", ["auto", "hist"])
        "scale_pos_weight": trial.suggest_float("scale_pos_weight", 1.0, 3.0),
    }
    model = XGBClassifier(
        **params,
        objective="binary:logistic",
        eval_metric="auc",
        use_label_encoder=False,
        random_state=42,
        n_jobs=-1
    )

    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model,X_train, y_train, scoring='roc_auc', cv=cv)

    return scores.mean()

In [ ]:
#Estudio para maximizar
study = optuna.create_study(direction="maximize")

#Búsqueda
study.optimize(objective, n_trials=50, show_progress_bar=True)

print("Best trial:", study.best_trial.number)
print("Best value (mean AUC):", study.best_value)
print("Best hyperparams:", study.best_params)

In [ ]:
import pickle
with open("XGBoost_Optuna_Study_v1_space_reducido_mas_hiperpara.pkl", "wb") as f:
    pickle.dump(study, f)
with open("XGBoost_Optuna_Study_v1_space_reducido_mas_hiperpara.pkl", "rb") as f:
    loaded_study = pickle.load(f)

In [ ]:
from tabulate import tabulate

trials_df = loaded_study.trials_dataframe()

df_sorted = trials_df.sort_values(by="value", ascending=False)
best_10 = df_sorted.head(10)

print(tabulate(best_10, headers="keys", tablefmt="psql"))

In [5]:
def objective_completo(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 1500, 2000, step=50),
        "max_bin": trial.suggest_int("max_bin", 270, 350, step=10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.03, log=True),
        "max_depth": trial.suggest_int("max_depth", 1, 6),
        "gamma": trial.suggest_float("gamma", 0, 1.5),
        "min_child_weight": trial.suggest_int("min_child_weight", 17, 25),
        "subsample": trial.suggest_float("subsample", 0.75, 0.9),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 0.6),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 1.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 2.2, log=True),
        "tree_method": trial.suggest_categorical("tree_method", ["auto", "hist"]),
        "scale_pos_weight": trial.suggest_float("scale_pos_weight", 1.0, 2.0),
    }
    model = XGBClassifier(
        **params,
        objective="binary:logistic",
        eval_metric="auc",
        use_label_encoder=False,
        random_state=42,
        n_jobs=-1
    )

    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model,X_train, y_train, scoring='roc_auc', cv=cv)

    return scores.mean()

In [ ]:
#Estudio para maximizar
study_completo = optuna.create_study(direction="maximize")

#Búsqueda
study_completo.optimize(objective_completo, n_trials=100, show_progress_bar=True)

print("Best trial:", study_completo.best_trial.number)
print("Best value (mean AUC):", study_completo.best_value)
print("Best hyperparams:", study_completo.best_params)

[I 2025-03-06 19:46:34,125] A new study created in memory with name: no-name-609c031c-d878-476e-8773-f463cd19fb7f


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2025-03-06 20:17:48,905] Trial 0 finished with value: 0.7872607013603694 and parameters: {'n_estimators': 1850, 'max_bin': 320, 'learning_rate': 0.011635182197463066, 'max_depth': 3, 'gamma': 0.9559734825886956, 'min_child_weight': 17, 'subsample': 0.8006007028312263, 'colsample_bytree': 0.5857692984725534, 'reg_alpha': 3.599478939814868e-08, 'reg_lambda': 8.696776925638852e-05, 'tree_method': 'auto', 'scale_pos_weight': 1.6451510057243541}. Best is trial 0 with value: 0.7872607013603694.
[I 2025-03-06 20:40:25,641] Trial 1 finished with value: 0.782503502520601 and parameters: {'n_estimators': 1600, 'max_bin': 320, 'learning_rate': 0.01564479336356769, 'max_depth': 2, 'gamma': 0.8953749094109904, 'min_child_weight': 21, 'subsample': 0.8222730815144866, 'colsample_bytree': 0.5821538323899231, 'reg_alpha': 0.2662914789801197, 'reg_lambda': 0.001740508510301081, 'tree_method': 'auto', 'scale_pos_weight': 1.3495086017133466}. Best is trial 0 with value: 0.7872607013603694.
[I 2025-03-0

In [ ]:
import pickle
with open("XGBoost_Optuna_Study_v7_completo_1183_features.pkl", "wb") as f:
    pickle.dump(study_completo, f)
#with open("/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/XGBoost_Optuna_Study_v7_completo.pkl", "rb") as f:
#    loaded_study_completo = pickle.load(f)

In [ ]:
from tabulate import tabulate

trials_df = loaded_study_completo.trials_dataframe()

df_sorted = trials_df.sort_values(by="value", ascending=False)
best_10 = df_sorted.head(10)

print(tabulate(best_10, headers="keys", tablefmt="psql"))

In [ ]:
best_row = best_10.iloc[0]

best_params = best_row.filter(like="params_").to_dict()

best_params_clean = {
    k.replace("params_", ""): v 
    for k, v in best_params.items()
}

final_params = {
    **best_params_clean,
    "objective": "binary:logistic",
    "eval_metric": "auc",
    "use_label_encoder": False,
    "random_state": 42
}

XGB_model = XGBClassifier(**final_params)

XGB_model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import roc_auc_score

y_pred_proba = XGB_model.predict_proba(X_test)[:, 1]
auc_test = roc_auc_score(y_test, y_pred_proba)
print(f"AUC en test: {auc_test:.4f}")


In [ ]:
Primera prueba:
|    |   number |    value | datetime_start             | datetime_complete          | duration               |   params_colsample_bytree |   params_gamma |   params_learning_rate |   params_max_depth |   params_min_child_weight |   params_n_estimators |   params_reg_alpha |   params_reg_lambda |   params_scale_pos_weight |   params_subsample | state    |
|----+----------+----------+----------------------------+----------------------------+------------------------+---------------------------+----------------+------------------------+--------------------+---------------------------+-----------------------+--------------------+---------------------+---------------------------+--------------------+----------|
| 35 |       35 | 0.766467 | 2025-02-23 19:05:51.648664 | 2025-02-23 19:23:42.992309 | 0 days 00:17:51.343645 |                  0.609149 |        2.79174 |             0.016566   |                  7 |                        14 |                   950 |        2.98383e-07 |         0.000771683 |                   1.7469  |           0.753771 | COMPLETE 
Tras una segunda prueba añadiendo más HP y acotando algunos rangos obtenemos:

|    |   number |    value | datetime_start             | datetime_complete          | duration               |   params_colsample_bytree |   params_gamma |   params_learning_rate |   params_max_bin |   params_max_depth |   params_min_child_weight |   params_n_estimators |   params_n_jobs |   params_reg_alpha |   params_reg_lambda |   params_scale_pos_weight |   params_subsample | params_tree_method   | state    |
|----+----------+----------+----------------------------+----------------------------+------------------------+---------------------------+----------------+------------------------+------------------+--------------------+---------------------------+-----------------------+-----------------+--------------------+---------------------+---------------------------+--------------------+----------------------+----------|
| 42 |       42 | 0.767633 | 2025-02-24 17:35:27.403636 | 2025-02-24 17:49:11.175013 | 0 days 00:13:43.771377 |                  0.58057  |       0.476025 |              0.0170753 |              300 |                  5 |                        23 |                  1800 |               2 |        2.30559e-07 |         2.39006e-05 |                   1.25498 |           0.788019 | hist                 | COMPLETE |

Y si realizamos una prueba en el test con estos valores obtenemos:
-AUC en test: 0.7717


-He detenido las pruebas porque me estaba yendo demasiado lento:
[I 2025-02-26 12:07:34,193] Trial 2 finished with value: 0.7938360589390858 and parameters: {'n_estimators': 1700, 'max_bin': 320, 'learning_rate': 0.016029922136908095, 'max_depth': 6, 'gamma': 0.8414416893725896, 'min_child_weight': 21, 'subsample': 0.8023911163856219, 'colsample_bytree': 0.5298608589445156, 'reg_alpha': 1.8193616624679977e-06, 'reg_lambda': 1.0784883537935654e-06, 'tree_method': 'auto', 'scale_pos_weight': 1.1284697519313376}. Best is trial 2 with value: 0.7938360589390858.

Tras realizar una búsqueda sobre los mejores HP de XGBoost hemos obtenido:
|    |   number |    value | datetime_start             | datetime_complete          | duration               |   params_colsample_bytree |   params_gamma |   params_learning_rate |   params_max_bin |   params_max_depth |   params_min_child_weight |   params_n_estimators |   params_reg_alpha |   params_reg_lambda |   params_scale_pos_weight |   params_subsample | params_tree_method   | state    |
|----+----------+----------+----------------------------+----------------------------+------------------------+---------------------------+----------------+------------------------+------------------+--------------------+---------------------------+-----------------------+--------------------+---------------------+---------------------------+--------------------+----------------------+----------|
| 88 |       88 | 0.794317 | 2025-03-02 07:37:37.702247 | 2025-03-02 08:23:05.401147 | 0 days 00:45:27.698900 |                  0.552899 |       0.382194 |              0.0175943 |              320 |                  6 |                        21 |                  1950 |          0.57723   |         0.00450461  |                   1.06067 |           0.765582 | auto                 | COMPLETE |

Y sobre AUC hemos obtenido: |
